使用tensorflow產生.h5檔

In [51]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, roc_auc_score

# pictures need to be in subdirectories!
train_dir = 'reference/'
test_dir = 'test/'

# Define image dimensions
img_width, img_height = 64, 64

# Prepare data generators
train_datagen = ImageDataGenerator(rescale=1./255)      # rescale pixel values from 0-255(8 bits) to range(0, 1)
train_generator = train_datagen.flow_from_directory(
    train_dir,                                          # automatically labels cats/ to be 0, and dogs/ to be 1
    target_size=(img_width, img_height),
    batch_size=10,                                      # 10 images at a time
    class_mode='binary')                                # cats(0) vs dogs(1)

# Define the CNN model
model = Sequential([
    # 32 filters(3*3)
    # ReLU = rectified linear unit = non-linearity (rectify = 修正)
    # shape of 64*64 with 3 channels(RGB)
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),   # fully connected layer
    Dense(1, activation='sigmoid')  # output layer
])

# Compile the model
# small learning rate leads to more precise convergence, but slower training process
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# steps_per_epoch * epochs = total number of training pictures
history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=10   # Generally increasing epochs leads to higher accuracy
)

# Evaluate the model
y_true = train_generator.classes                                # receive class labels(cats = 0, dogs = 1)
y_pred = model.predict(train_generator)                         # predict training dataset
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred.ravel()]

# Confusion Matrix and AUROC
conf_matrix = confusion_matrix(y_true, y_pred)
auroc = roc_auc_score(y_true, y_pred)

# Print confusion matrix and AUROC
print("Confusion Matrix:\n", conf_matrix)
print("AUROC:", auroc)

# Save model weights
model.save_weights('hw4.h5')

# Load model weights and predict on new data
model.load_weights('hw4.h5')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False)

predictions = model.predict(test_generator)
predictions = [1 if prob > 0.5 else 0 for prob in predictions.ravel()]

# Output predictions
for i, pred in enumerate(predictions):
    print(f'pic{i+1}.jpg - {"Dog" if pred == 1 else "Cat"}')


Found 100 images belonging to 2 classes.


Epoch 1/10
10/10 [==============================] - 1s 34ms/step - loss: 0.7584 - accuracy: 0.5300
Epoch 2/10
10/10 [==============================] - 0s 34ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 0s 33ms/step - loss: 0.6790 - accuracy: 0.6600
Epoch 4/10
10/10 [==============================] - 0s 33ms/step - loss: 0.6705 - accuracy: 0.5900
Epoch 5/10
10/10 [==============================] - 0s 34ms/step - loss: 0.5978 - accuracy: 0.7500
Epoch 6/10
10/10 [==============================] - 0s 34ms/step - loss: 0.5449 - accuracy: 0.7700
Epoch 7/10
10/10 [==============================] - 0s 33ms/step - loss: 0.4535 - accuracy: 0.8300
Epoch 8/10
10/10 [==============================] - 0s 35ms/step - loss: 0.3655 - accuracy: 0.9000
Epoch 9/10
10/10 [==============================] - 0s 36ms/step - loss: 0.3246 - accuracy: 0.8800
Epoch 10/10
10/10 [==============================] - 0s 24ms/step
Confusion Matrix:
 [[31 19]
 [20 30]]
AUROC

Confusion Matrix:
 [[31 19]
 [20 30]]
AUROC: 0.61